In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import pickle
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH10-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH10-Retriever


In [4]:
with open("results/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [5]:
split_documents

[Document(metadata={'page': 1, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': []}, page_content='OECD\n\nOECD\n\nOECD Economic Outlook, Interim Report\n=====================================\n\n  \n\nFinding the Right Balance in Uncertain Times\n============================================\n\nSeptember 2025\n=============='),
 Document(metadata={'page': 3, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': []}, page_content='OECD Economic Outlook, Interim Report\n=====================================\n\nOECD Economic Outlook, Interim Report\n=====================================\n\nFinding the Right Balance in Uncertain Times\n============================================\n\nSeptember 2025\n==============\n\nOECD  \nBETTER POLICIES FOR BETTER LIVES'),
 Document(metadata={'page': 4, 'image_id': [], 'image_path': [], 'text_summary': [{'page': 4, 'summary': '요약 (한국어)\n\n- 본 문서는 OECD 사무총장의 책임 하에 발행되었으며, 문서에 담긴 의견과 논지는 OECD 회원국들의 공식 견해를 반드시 반영하지 않는다

In [6]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [7]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [8]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [9]:
# bm25 retriever와 faiss retriever를 초기화합니다.
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 1 

In [10]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

In [11]:
# 검색 결과 문서를 가져옵니다.
query = "Korea"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: | France | 2.3 | 1.8 | 0.2 | 1.7 | 0.0 |
| Italy | 2.2 | 1.9 | 0.4 | 1.8 | 0.1 |
| Spain2 | 2.8 | 2.3 | 0.0 | 1.9 | -0.1 |
| Japan | 2.0 | 1.7 | -0.2 | 2.0 | 0.0 |
| Korea | 2.2 | 2.0 | 0.0 | 2.0 | 0.0 |
| Mexico | 4.1 | 4.1 | 0.5 | 3.5 | 0.3 |
| Turkiye | 59.8 | 33.5 | 1.2 | 18.9 | 0.3 |
| United Kingdom | 3.7 | 3.7 | 0.6 | 2.7 | 0.4 |
| United States | 2.8 | 2.9 | -0.6 | 3.0 | 0.2 |
| South Africa | 4.2 | 3.3 | 0.0 | 3.7 | -0.6 |

Content: I 5

I 5

[BM25 Retriever]
Content: | France | 2.3 | 1.8 | 0.2 | 1.7 | 0.0 |
| Italy | 2.2 | 1.9 | 0.4 | 1.8 | 0.1 |
| Spain2 | 2.8 | 2.3 | 0.0 | 1.9 | -0.1 |
| Japan | 2.0 | 1.7 | -0.2 | 2.0 | 0.0 |
| Korea | 2.2 | 2.0 | 0.0 | 2.0 | 0.0 |
| Mexico | 4.1 | 4.1 | 0.5 | 3.5 | 0.3 |
| Turkiye | 59.8 | 33.5 | 1.2 | 18.9 | 0.3 |
| United Kingdom | 3.7 | 3.7 | 0.6 | 2.7 | 0.4 |
| United States | 2.8 | 2.9 | -0.6 | 3.0 | 0.2 |
| South Africa | 4.2 | 3.3 | 0.0 | 3.7 | -0.6 |

[FAISS Retriever]
Content: I 5

I 5



In [12]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [13]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
chain = (
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "한국의 경제 관련 내용 있어?"
response = chain.invoke(question)
print(response)

네 있습니다. 제공된 자료에서는 OECD 중간 전망표에 한국의 핵심인플레이션 전망이 나와 있습니다.

- 한국 핵심인플레이션(식료품·에너지 제외) 전망: 2024년 2.2% → 2025년 2.0% → 2026년 2.0% (6월 전망 대비 변경 없음).

그 밖에 제공된 문서에는 한국에 대한 추가 상세 언급은 없습니다. 더 자세한 정보를 원하시면 어떤 항목(성장률, 고용, 수출 등)을 찾을지 알려주세요.
